# Eyes closed masking file

In [2]:
import pandas as pd
import numpy as np
import h5py

dir = "E:/ChristianMusaeus/Data/Eyes_closed_marked"
hf = h5py.File(f"{dir}/eyesclosedmarked.mat", "r")
print(hf.keys())
print(hf["files"])
# extract data from files group

df = pd.DataFrame(hf["data"][:]).T
# keep only the first and the last 29 rows, since this is where there is eyes open data. And drop the 4th last patient since too little data.
keep = np.concatenate([np.array([0]), np.arange(51, 80)])
df = df.iloc[keep, :]
# drop the 4th last patient
df = df.drop(df.index[-4])


print(df.head())
print(df.shape)


# # Convert to a pandas dataframe
# data = np.array(new_data).T
# print(df.head())


<KeysViewHDF5 ['#refs#', '#subsystem#', 'EEG', 'data', 'files', 'ii']>
<HDF5 group "/files" (6 members)>
    0     1     2     3     4     5     6     7     8     9     ...  5593  \
0    1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   0.0   
51   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   0.0   
52   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   0.0   
53   1.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
54   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   0.0   

    5594  5595  5596  5597  5598  5599  5600  5601  5602  
0    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
51   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
52   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
53   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
54   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 5603 columns]
(29, 5603)


# Eyes closed data

In [ ]:
import mne
import os

newdata = []
dir = "E:/ChristianMusaeus/Data/Eyes_closed_marked"
for i in range(10002, 10213 + 1):
    file_name = f"{i}_p01_epoched_60EpochsMarked.set"
    if os.path.isfile(os.path.join(dir, file_name)):
        data = mne.io.read_epochs_eeglab(f"{dir}/{file_name}",verbose=False)
        newdata.append(data.get_data())
print(len(newdata))
print(newdata[0].shape[0])
print(newdata[1].shape)




In [4]:
#save newdata as a pickle file
import pickle
with open(f"{dir}/closeddata.pkl", "wb") as f:
    pickle.dump(newdata, f)

#load newdata from pickle file



# Create ID list to keep only relevant data

In [5]:
import pickle
with open(f"{dir}/closeddata.pkl", "rb") as f:
    closeddata = pickle.load(f)

keep = np.concatenate([np.array([0]), np.arange(51, 80)])
# keep only the lists that are in keep array
closeddata = [data for i, data in enumerate(closeddata) if i in keep]
#delete 4th last list
del closeddata[-4]
print(len(closeddata))


29


In [6]:
import numpy as np

#create a list of patient ids from one to 80 with each occuring as many timesa as closeddata[i].shape[0]

patient_ids = np.arange(0, 29)
patient_ids = np.repeat(patient_ids, [data.shape[0] for data in closeddata])
print(patient_ids)
#couint each entryie of each patient id
from collections import Counter
print(Counter(patient_ids))


[ 0  0  0 ... 28 28 28]
Counter({np.int64(20): 4234, np.int64(7): 3618, np.int64(11): 3371, np.int64(10): 3043, np.int64(3): 2736, np.int64(8): 2701, np.int64(18): 2599, np.int64(17): 2592, np.int64(4): 2541, np.int64(9): 2257, np.int64(23): 2212, np.int64(5): 2051, np.int64(14): 1994, np.int64(6): 1911, np.int64(28): 1911, np.int64(22): 1879, np.int64(13): 1864, np.int64(1): 1859, np.int64(2): 1847, np.int64(15): 1824, np.int64(19): 1819, np.int64(12): 1816, np.int64(25): 1815, np.int64(27): 1810, np.int64(26): 1809, np.int64(0): 1807, np.int64(16): 1683, np.int64(21): 1372, np.int64(24): 816})


# Trim lists for eyes closed

In [ ]:
# convert df to a list of numpy arrays
labeldata = []
for i in range(df.shape[0]):
    labeldata.append(df.iloc[i].values)
print(len(labeldata))
print(labeldata[1].shape)

from collections import Counter


# Count occurrences of each number in list 'a'
count_a = Counter(patient_ids)

# Create a new list of lists where each sublist is trimmed to the count of its corresponding number
trimmed_lists = []
for num in range(29):  # Assuming numbers are from 0 to 28
    occurrences = count_a[num]
    trimmed_list = labeldata[num][:occurrences]
    trimmed_lists.append(trimmed_list)


# Print amount of zeros in each list
for i, lst in enumerate(trimmed_lists):
    print(f"number of 0 entries in trimmed_list: {np.sum(trimmed_list == 0)}")
# looks good, 60 in each.



# For each row in closeddata, keep only the epochs where the label is 0 in trimmed_lists
trimmed_closed_data = []
for i, data in enumerate(closeddata):
    print(f"shape of data: {data.shape}")
    labels = trimmed_lists[i]
    trimmed_data = data[labels == 0]
    trimmed_closed_data.append(trimmed_data)
    print(f"shape of trimmed_data: {trimmed_data.shape}")



# Print the result
# for i, data in enumerate(trimmed_closed_data):
#     # print(f"Data for {i}: {data[:5]}")
#     # print(len(data))

#save trimmed_closed_data as a pickle file
with open(f"{dir}/trimmed_closed_data.pkl", "wb") as f:
    pickle.dump(trimmed_closed_data, f)


# Eyes open

In [8]:
# load in opendata.pkl
import pickle
dir = "E:/ChristianMusaeus/Data/Eyes_closed_marked"
with open(f"{dir}/opendata.pkl", "rb") as f:
    opendata = pickle.load(f)
    
# drop 4th last patient
del opendata[-4]

# Load in masking file, trim it down to correct size and apply it to data file

In [ ]:
import pandas as pd
import numpy as np
import h5py

hf = h5py.File(f"{dir}/eyesopenmarked.mat", "r")
print(hf.keys())
print(hf["files"])
# extract data from files group

dfopen = pd.DataFrame(hf["data"][:]).T
dfopen = dfopen[:30]
dfopen = dfopen.drop(dfopen.index[-4])
print(dfopen.shape)

# convert df to a list of numpy arrays
labeldata = []
for i in range(dfopen.shape[0]):
    labeldata.append(dfopen.iloc[i].values)
print(len(labeldata))
print(labeldata[1].shape)


open_patient_ids = np.arange(0, 29)
open_patient_ids = np.repeat(open_patient_ids, [data.shape[0] for data in opendata])
print(open_patient_ids)
print(f" number of unique patient ids: {len(np.unique(open_patient_ids))}")

#couint each entryie of each patient id
from collections import Counter
print(Counter(open_patient_ids))

# Count occurrences of each number in list 'a'
count_a = Counter(open_patient_ids)


# Create a new list of lists where each sublist is trimmed to the count of its corresponding number
trimmed_lists = []
for i, num in enumerate(np.unique(open_patient_ids)):  # Assuming numbers are from 0 to 
    occurrences = count_a[num]
    trimmed_list = labeldata[i][:occurrences]
    trimmed_lists.append(trimmed_list)


# For each row in opendata, keep only the epochs where the label is 0 in trimmed_lists
trimmed_open_data = []
zeros_in_labls = []
for i, data in enumerate(opendata):
    print(f"shape of data: {data.shape}")
    labels = trimmed_lists[i]
    trimmed_data = data[labels == 0]
    print(f" number of zeros in labels: {np.sum(labels == 0)}")
    zeros_in_labls.append(np.sum(labels == 0))
    trimmed_open_data.append(trimmed_data)
    print(f"shape of trimmed_data: {trimmed_data.shape}")



# create open_patients_ids from trimmed lists
open_patient_ids = np.arange(0, 29)
open_patient_ids = np.repeat(open_patient_ids, [data.shape[0] for data in trimmed_open_data])
print(open_patient_ids)
print(f" number of unique patient ids: {len(np.unique(open_patient_ids))}")


# save trimmed_open_data as a pickle file
with open(f"{dir}/trimmed_open_data.pkl", "wb") as f:
    pickle.dump(trimmed_open_data, f)



# Create one big dataframe 

In [4]:
import numpy as np
print(np.__version__)

2.1.3


In [5]:
# load in trimmed_open_data.pkl and trimmed_closed_data.pkl and convert both to dataframes and stack them vertically
import pickle
import pandas as pd
dir = "E:/ChristianMusaeus/Data/Eyes_closed_marked"
with open(f"{dir}/trimmed_open_data.pkl", "rb") as f:
    trimmed_open_data = pickle.load(f)

with open(f"{dir}/trimmed_closed_data.pkl", "rb") as f:
    trimmed_closed_data = pickle.load(f)


# convert trimmed_closed_data to a dataframe
closed_df = pd.DataFrame()
for i, data in enumerate(trimmed_closed_data):
    data = data.reshape(60, -1)
    df = pd.DataFrame(data)
    closed_df = pd.concat([closed_df, df], axis=0)
closed_df["label"] = 1

open_df = pd.DataFrame()
for i, data in enumerate(trimmed_open_data):
    data = data.reshape(data.shape[0],-1)
    df = pd.DataFrame(data)
    open_df = pd.concat([open_df, df], axis=0)
open_df["label"] = 0

# stack the two dataframes vertically
df = pd.concat([closed_df, open_df], axis=0)
print(df.head(), df.shape)

# save df as a pickle file
with open(f"{dir}/df_EOandEC.pkl", "wb") as f:
    pickle.dump(df, f)

          0         1         2         3         4         5             6  \
0 -0.000004 -0.000009 -0.000012 -0.000010 -0.000014 -0.000020 -1.990352e-05   
1 -0.000005 -0.000012 -0.000015 -0.000020 -0.000026 -0.000026 -2.479350e-05   
2  0.000022  0.000018  0.000013  0.000008  0.000003  0.000001  9.871100e-07   
3  0.000007  0.000003 -0.000001 -0.000004 -0.000007 -0.000008 -6.191659e-06   
4  0.000013  0.000013  0.000012  0.000012  0.000010  0.000010  1.009186e-05   

              7         8             9  ...          3791      3792  \
0 -2.007265e-05 -0.000019 -1.404443e-05  ... -1.092016e-05 -0.000010   
1 -2.346011e-05 -0.000018 -1.718349e-05  ...  1.259601e-06  0.000002   
2 -2.112761e-07 -0.000002 -1.798665e-06  ... -6.569762e-06 -0.000005   
3 -4.011699e-06 -0.000002  4.808684e-07  ... -3.040970e-06 -0.000005   
4  8.046945e-06  0.000007  4.599984e-06  ... -8.812295e-07  0.000002   

           3793      3794      3795      3796      3797          3798  \
0 -6.451550e-06 -0.

# Create Patient ID's

In [10]:
closed_patient_ids = np.repeat(np.arange(0, 29), 60)

print(len(closed_patient_ids), len(open_patient_ids))
patient_ids = np.concatenate([closed_patient_ids, open_patient_ids])

# attach patient ids to df
df["patient_id"] = patient_ids
print(df.shape)

print(df.head())


1740 1475
(3215, 3802)
          0         1         2         3         4         5             6  \
0 -0.000004 -0.000009 -0.000012 -0.000010 -0.000014 -0.000020 -1.990352e-05   
1 -0.000005 -0.000012 -0.000015 -0.000020 -0.000026 -0.000026 -2.479350e-05   
2  0.000022  0.000018  0.000013  0.000008  0.000003  0.000001  9.871100e-07   
3  0.000007  0.000003 -0.000001 -0.000004 -0.000007 -0.000008 -6.191659e-06   
4  0.000013  0.000013  0.000012  0.000012  0.000010  0.000010  1.009186e-05   

              7         8             9  ...      3792          3793  \
0 -2.007265e-05 -0.000019 -1.404443e-05  ... -0.000010 -6.451550e-06   
1 -2.346011e-05 -0.000018 -1.718349e-05  ...  0.000002  8.180189e-07   
2 -2.112761e-07 -0.000002 -1.798665e-06  ... -0.000005 -1.266182e-07   
3 -4.011699e-06 -0.000002  4.808684e-07  ... -0.000005 -4.692287e-06   
4  8.046945e-06  0.000007  4.599984e-06  ...  0.000002  2.542886e-06   

       3794      3795      3796      3797          3798      3799  la

# Scale data patientwise(global patient parameters)

In [11]:

from sklearn.preprocessing import MinMaxScaler
import numpy as np

y = df["label"].values
patient_ids = df["patient_id"].values

# Normalize per patient (within training and test sets)
print(f"before scaling: {df.head(2)}")
df = df.drop(columns=["label", "patient_id"])  # Drop the label and patient id columns
df_norm = []
for patient_id in np.unique(patient_ids):
    patient_data = df[patient_ids == patient_id]
    scaler = MinMaxScaler(feature_range=(-1, 1))
    patient_data_scaled = scaler.fit_transform(patient_data)
    df_norm.append(patient_data_scaled)

df_norm = np.concatenate(df_norm, axis=0)
# Add labels back
df_norm = np.concatenate([df_norm, y.reshape(-1, 1)], axis=1)
# Add patient ids back
df_norm = np.concatenate([df_norm, patient_ids.reshape(-1, 1)], axis=1)
print(f"after scaling: {df_norm[:2]}")
print(f"min and max: {np.min(df_norm)}, {np.max(df_norm[:, :-2])}")

# save df_norm as a pickle file
with open(f"{dir}/df_EOandEC_norm.pkl", "wb") as f:
    pickle.dump(df_norm, f)

# save in a differnt format also
df = pd.DataFrame(df_norm)
df.to_csv(f"{dir}/df_EOandEC_norm.csv", index=False)




before scaling:           0         1         2        3         4         5         6  \
0 -0.000004 -0.000009 -0.000012 -0.00001 -0.000014 -0.000020 -0.000020   
1 -0.000005 -0.000012 -0.000015 -0.00002 -0.000026 -0.000026 -0.000025   

          7         8         9  ...      3792          3793      3794  \
0 -0.000020 -0.000019 -0.000014  ... -0.000010 -6.451550e-06 -0.000002   
1 -0.000023 -0.000018 -0.000017  ...  0.000002  8.180189e-07 -0.000004   

       3795      3796      3797      3798      3799  label  patient_id  
0  0.000002  0.000005  0.000007  0.000007  0.000006      1           0  
1 -0.000009 -0.000012 -0.000012 -0.000010 -0.000007      1           0  

[2 rows x 3802 columns]
after scaling: [[-0.22054073 -0.23624156 -0.17988051 ...  0.60395375  1.
   0.        ]
 [-0.23374289 -0.28206085 -0.22675689 ... -0.11500413  1.
   0.        ]]
min and max: -1.0, 1.0000000000000004


In [12]:

# check min and max values
print(f"min: {np.min(df_norm)}, max: {np.max(df_norm[:, :-2])}")


min: -1.0, max: 1.0000000000000004


# Fourier Transform

In [13]:
from scipy.signal import stft
import numpy as np

reshaped_data = df_norm[:, :-2]

fs = 200  # Sampling frequency (in Hz)
nperseg = fs/2  # Length of each segment for STFT (typically a power of 2)

# create empty list for the spectrograms, each channel will have its own list
spectrograms = [[] for _ in range(reshaped_data.shape[0])]
print(f"shape of spectrograms: {len(spectrograms)}")

for i, epoch in enumerate(reshaped_data): # loop over datarows
    frequencies, times, Zxx = stft(epoch, fs, nperseg=nperseg)
    spectrogram = np.abs(Zxx)  # Shape: (n_freq_bins, n_time_segments)
    # print(f"shape of spectrogram: {spectrogram.shape}")

    # Flatten the spectrogram of this channel
    spectrogram_flattened = spectrogram.flatten()  # Flatten to 1D array
    
    # Append to list
    spectrograms[i] = spectrogram_flattened

print(len(spectrograms[0]))

# save spectrograms as a pickle file
spectrograms = np.array(spectrograms)
df = pd.DataFrame(spectrograms)
df["label"] = y
df["patient_id"] = patient_ids
print(df.head())

with open(f"{dir}/hospital_spectrograms.pkl", "wb") as f:
    pickle.dump(spectrograms, f)

# save as a csv file
df.to_csv(f"{dir}/hospital_spectrograms.csv", index=False)



shape of spectrograms: 3215
3927
          0         1         2         3         4         5         6  \
0  0.186564  0.475278  0.505854  0.306700  0.255097  0.557123  0.555072   
1  0.131151  0.520671  0.447478  0.456006  0.052131  0.583297  0.527655   
2  0.123682  0.148802  0.332623  0.198912  0.240235  0.298129  0.357936   
3  0.168667  0.463728  0.337087  0.306425  0.304572  0.543767  0.367835   
4  0.143401  0.420038  0.098637  0.138659  0.324467  0.524166  0.083871   

          7         8         9  ...      3919      3920      3921      3922  \
0  0.407374  0.221913  0.021099  ...  0.001207  0.002463  0.000491  0.005581   
1  0.536725  0.081143  0.052932  ...  0.002277  0.001954  0.001542  0.003579   
2  0.325636  0.169057  0.191902  ...  0.002602  0.001009  0.000062  0.007482   
3  0.455260  0.288131  0.157854  ...  0.001342  0.001138  0.001100  0.000432   
4  0.324291  0.224512  0.130821  ...  0.001786  0.002207  0.000786  0.001823   

       3923      3924      3925    

# Log Reg Loop

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import LeaveOneGroupOut

accs = []

# Initialize Leave-One-Group-Out cross-validator
logo = LeaveOneGroupOut()

patient_ids = df_norm[:, -1]
y = df_norm[:, -2]
data = df_norm[:, :-2]

# Train-test split using Leave-One-Group-Out
for i, (train_index, test_index) in enumerate(logo.split(data, y, groups=patient_ids)):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the logistic regression model
    model = LogisticRegression(max_iter=1000, verbose=1,class_weight="balanced")
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accs.append(accuracy)
    print(f'Model accuracy for Patient {i}: {accuracy*100:.2f}%')

# Print the average accuracy over all patients
print(f'Average accuracy across patients: {np.mean(accs)*100:.2f}%')


Model accuracy for Patient 0: 90.29%
Model accuracy for Patient 1: 99.43%
Model accuracy for Patient 2: 100.00%
Model accuracy for Patient 3: 98.12%
Model accuracy for Patient 4: 100.00%
Model accuracy for Patient 5: 97.26%
Model accuracy for Patient 6: 100.00%
Model accuracy for Patient 7: 97.95%
Model accuracy for Patient 8: 100.00%
Model accuracy for Patient 9: 98.70%
Model accuracy for Patient 10: 99.07%
Model accuracy for Patient 11: 96.70%
Model accuracy for Patient 12: 100.00%
Model accuracy for Patient 13: 99.03%
Model accuracy for Patient 14: 100.00%
Model accuracy for Patient 15: 100.00%
Model accuracy for Patient 16: 96.39%
Model accuracy for Patient 17: 100.00%
Model accuracy for Patient 18: 100.00%
Model accuracy for Patient 19: 99.36%
Model accuracy for Patient 20: 98.10%
Model accuracy for Patient 21: 98.94%
Model accuracy for Patient 22: 99.09%
Model accuracy for Patient 23: 100.00%
Model accuracy for Patient 24: 100.00%
Model accuracy for Patient 25: 99.19%
Model accur